In [4]:
# Setup environment
import os
import glob
import subprocess
import sys
import time
from scipy.io import netcdf as nc

import matplotlib.pyplot as plt
from os.path import expanduser

# Define output path
home = expanduser("~")
output_dir = os.path.join(home,'scratch/elm_testruns')  # main output directory.
os.makedirs(os.path.dirname(output_dir), exist_ok=True)
print(output_dir)

/Users/sserbin/scratch/elm_testruns


In [5]:
# Grab latest ELM docker container from Docker hub
source = "fasstsimulation/elm-builds"
docker_container_tag = "elm_v2-for-ngee"
container_name = ''.join([source,":",docker_container_tag])
print(container_name)
docker_pull_command = ''.join(["docker pull ",source,":",docker_container_tag])
os.system(docker_pull_command)

fasstsimulation/elm-builds:elm_v2-for-ngee
elm_v2-for-ngee: Pulling from fasstsimulation/elm-builds
Digest: sha256:053d765889c59a9d0c3937e36e50f125ad3be83916b6651e450867e845dc341d
Status: Image is up to date for fasstsimulation/elm-builds:elm_v2-for-ngee
docker.io/fasstsimulation/elm-builds:elm_v2-for-ngee


0

In [7]:
# Create docker volumes for output
elmdata_name = "elmdata"
elmoutput_name = "elmoutput"
run_command1 = ''.join(['docker volume create ', elmdata_name])
run_command2 = ''.join(['docker volume create ', elmoutput_name])
#print(run_command)
os.system(run_command1)
os.system(run_command2)

elmdata
elmoutput


0

In [12]:
# Define run options for OLMT
#docker run -t -i --hostname=docker --user modeluser -v elmdata:/inputdata -v elmoutput:/output fasstsimulation/elm-builds:elm_v2-for-ngee /scripts/OLMT_docker_example.sh
hostname="docker"
user="modeluser"
elmdata="elmdata"
elmoutput="elmoutput"

elm_run_command = ''.join(['docker run -t -i --hostname ', hostname,' --user ', user])
print(elm_run_command)

docker run -t -i --hostname docker --user modeluser


In [ ]:
# Run the model simulation
# Create new case
start = time.time()
# run the docker container
os.system(elm_run_command)
# get runtime
end = time.time()
#Print runtime
print(end - start)